### MSDS  7337-404 - Natural Language Processing ###

### Homework5 ###
**Shah Dhyan**

*** Question 1 ***

****Compile a list of static links (permalinks) to individual user movies reviews from one particular****
****website. This will be your working dataset for this assignment, as well as for assignments 7 and 8****
****which together will make up your semester project.**** 

****a.****

****It does not matter if you use a crawler or if you manually collect the links, but you will****
****need at least 100 movie review links. Note that, as of tis writing, the robots.txt file of****
****IMDB.com allows the crawling of user reviews.****

****b.**** 

****Each link should be to a web page that has only one use review of only one movie, e.g., the****
****user review permalinks on the IMDB site****

****c.****

****Choose reviews of movies that are all in the same genere, e.g., sic-fi, mystery, romance, superhero etc.****

****d.****

****Make sure your collection includes reviews of several movies in your chosen genre and that it includes a****
****mix of negative and positive reviews****

In [1]:
import bs4; print( 'bs4 ' + bs4.__version__)
from bs4 import BeautifulSoup, SoupStrainer

import nltk; print( 'nltk ' + nltk.__version__)
from nltk import word_tokenize, pos_tag, RegexpParser;
from nltk.tokenize import sent_tokenize

import re; print('re ' + re.__version__)
import requests; print('requests ' + requests.__version__)

bs4 4.8.0
nltk 3.4.5
re 2.2.1
requests 2.22.0


In [79]:
# Source links for Top Alltime movie reviews 
review_urls = {
    'The Shawshank Redemption': 'https://www.imdb.com/title/tt0111161/reviews?ref_=tt_urv3',
    'The Godfather': 'https://www.imdb.com/title/tt0068646/reviews?ref_=tt_urv',
    'The Dark Knight ': 'https://www.imdb.com/title/tt0468569/reviews?ref_=tt_urv',
    '12 Angry Men': 'https://www.imdb.com/title/tt0050083/reviews?ref_=tt_urv',
    'Schindlers List' : 'https://www.imdb.com/title/tt0108052/reviews?ref_=tt_urv',
    'The Lord of the Rings: The Return of the King ': 'https://www.imdb.com/title/tt0167260/reviews?ref_=tt_urv',
    'Reservoir Dogs ' :'https://www.imdb.com/title/tt0105236/reviews?sort=helpfulnessScore&dir=desc&ratingFilter=1',
}

In [66]:
def get_text_url(url):
    return requests.get(url).text
text = get_text_url(review_home_urls['The Shawshank Redemption'])

In [67]:
def get_all_html_links(html):
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer('a', href=True))
    urls = [str(tag.attrs['href']) for tag in tags]
    return urls
all_links = get_all_html_links(text)

In [68]:
def get_review_links(links):
    url_template = 'https://www.imdb.com{}'
   
    return [url_template.format(link) for link in links]

urls = get_review_links(all_links);

In [69]:
def relevent_link(link):
    if '/review/' in link:
        return True
    return False
def get_relevent_links(links):
    relevent_links = filter(relevent_link, all_links)
    unique_relevent_links = set(relevent_links)
    return list(unique_relevent_links)
relevent_urls = get_relevent_links(urls)
len(relevent_urls)

25

## Movies: ##

The Shawshank Redemption,

The Godfather, 

The Dark Knight, 

12 Angry Men,

Schindlers List, 

The Lord of the Rings: The Return of the King and 

Reservoir Dogs . 

Used various ratings between 1 and 10 reviews and store the review pages in a variable called review_urls and scrape all of the reviews and store all_reviews. Total 175 variety ratings reviews were utilized for analysis. 

In [70]:
def strain_content(name, attrs):
    if name == 'div' and dict(attrs).get('class', None) == 'content':
        return True
    return False
def clean_review_text(text):
    return re.split('\\n\\n\s+\d+ out of \d+', text)[0]
def get_review_from_url(url):
    html = get_text_from_url(url)
    tags = BeautifulSoup(html, 'html.parser', parse_only=SoupStrainer(strain_content))
    review = clean_review_text(tags.text)
    return review

In [71]:
def get_review_from_site(url):
    reviews = []

    reviews_home_text = get_text_from_url(url)
    all_links = get_all_links_from_html(reviews_home_text)
    relevent_links = get_relevent_links(all_links)

    
    
    
    review_urls = get_review_urls_from_links(relevent_links)
    for url in review_urls:
        reviews.append(get_review_from_url(url))
        # break
    return reviews

In [72]:
def get_reviews_from_all_sites():
    all_reviews = []
    review_titles = review_home_urls.keys()
    for title in review_titles:
        review_home_url = review_home_urls[title]
        all_reviews = all_reviews + get_review_from_site(review_home_url)
    return all_reviews

In [73]:
def pos_tagging_for_review(sentence):
    cleaned_review = sentence.lower()
    tokenized_review = word_tokenize(cleaned_review)
    return pos_tag(tokenized_review)
def get_chunking_for_sentence(sentence):
    tagged_sentence = pos_tagging_for_review(sentence)

    grammar = "NP: {<DT>?<JJ>*<NN>}"

    cp = nltk.RegexpParser(grammar)
    result = cp.parse(tagged_sentence)
    return result

def get_chunking_for_review(review):
    sentences = sent_tokenize(review)
    result = [get_chunking_for_sentence(sentence) for sentence in sentences]
    return result


In [74]:
def get_chunking_for_reviews(reviews):
    return [get_chunking_for_review(review) for review in reviews]

In [75]:
def get_NP_for_review(review):
    main_trees = get_chunking_for_review(review)
    subtrees = []
    for main_tree in main_trees:
        for subtree in main_tree.subtrees():
            if subtree.label() == 'NP':
                subtrees.append(subtree)
    return subtrees[1:]

### Each reviews are tokenized and tagged. Pass them through a parser. ###


In [76]:
def get_NP_for_reviews(reviews):
    return [get_NP_for_review(review) for review in reviews]

In [77]:
all_reviews = get_reviews_from_all_sites()

In [78]:
print(len(all_reviews))

175


In [82]:
NP_reviews = get_NP_for_reviews(all_reviews)

In [83]:
for noun_phrases in noun_phrases_reviews[0:6]:
    for i in range(2):
        print(noun_phrases[i])
    print()

(NP hope/NN)
(NP hope/NN)

(NP every/DT time/NN)
(NP i/NN)

(NP the/DT movie/NN)
(NP the/DT film/NN)

(NP the/DT shawshank/NN)
(NP redemption/NN)

(NP stand/NN)
(NP that/DT list.this/NN)

(NP this/DT movie/NN)
(NP title/NN)



### The output looks improved.

In [84]:
noun_phrases_reviews

[[Tree('NP', [('hope', 'NN')]),
  Tree('NP', [('hope', 'NN')]),
  Tree('NP', [('a', 'DT'), ('film', 'NN')]),
  Tree('NP', [('a', 'DT'), ('movie', 'NN')]),
  Tree('NP', [('a', 'DT'), ('lesson', 'NN')]),
  Tree('NP', [('life', 'NN')]),
  Tree('NP', [('a', 'DT'), ('story', 'NN')]),
  Tree('NP', [('freedom', 'NN')]),
  Tree('NP', [('freedom', 'NN')]),
  Tree('NP', [('isolation', 'NN')]),
  Tree('NP', [('rule', 'NN')]),
  Tree('NP', [('bigotry', 'NN')]),
  Tree('NP', [('hate', 'NN')]),
  Tree('NP', [('freeman', 'NN')]),
  Tree('NP', [('relationship', 'NN')]),
  Tree('NP', [('the', 'DT'), ('first', 'JJ'), ('moment', 'NN')]),
  Tree('NP', [('contact', 'NN')]),
  Tree('NP', [('a', 'DT'), ('wonderful', 'JJ'), ('performance', 'NN')]),
  Tree('NP', [('brooks.he', 'NN')]),
  Tree('NP', [('time', 'NN')]),
  Tree('NP', [('the', 'DT'), ('world', 'NN')]),
  Tree('NP', [('the', 'DT'), ('world', 'NN')]),
  Tree('NP', [('a', 'DT'), ('chance', 'NN')]),
  Tree('NP', [('stephen', 'NN')]),
  Tree('NP', [('st

Used movies from Drama genre which are all time top 100 movies. Used review that were recorded between 1 and 10 ratings, where 1 = Waste of Time and 10 = Must Watch ranking from users on IMDB.com. First, manually selected the "main review pages" for each movie's review and sorted for different rating reivies to have a balance ratings among the 175 review which then were iterated over each page for collection of links of reviews. 